In [2]:
import pyvista as pv
import numpy as np
from numpy import newaxis
import math
import glob
import keras
import os
import re
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam, SGD
import pathlib
import time
import datetime
import sklearn.preprocessing

%matplotlib inline

c:\Users\Vaishnav Bhaskaran\anaconda3\envs\GAN\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
fp_vtk = r'C:\Users\Vaishnav Bhaskaran\GAN\Data\VTK_Data\With_Hole\Aluminium'

### Data Preprocessing:

In [4]:
def list_to_array(inp_list, l_type):
    dummy_list = []

    if l_type != 'bc':
        for i in range(len(inp_list)):
            dummy_list.append(list(map(float, inp_list[i].split('\n')[0].split(','))))
    else:
        for i in range(len(inp_list)):
            dummy_list += list(map(float, inp_list[i].split('\n')[0].split(',')))
    
    return(np.array(dummy_list))

In [5]:
def get_inp(filepath):

    node_list = []
    element_list = []
    bc_enc = []
    bc_load = []
    node = False
    element = False
    bc = False
    ld = False

    fp0 = filepath.split('\\')
    fp0 = fp0[4:]
    fp0[-2] = str(fp0[-2][:-4])
    n0 = str(fp0[-1])[0:12]
    n1 = n0 + str('.inp')
    fp0.pop(-1)
    fp0.extend([n0, n1])
    fp1 = os.path.abspath(os.path.join(*fp0))

    f_attr = file_attr(fp1)

    with open(fp1) as inp_file:
        for lines in inp_file:
            if lines.startswith('*Node'):
                node = True
            if node:
                node_list.append(lines)
            if lines.startswith('*Element'):
                node = False
                element = True
            if element:
                element_list.append(lines)
            if lines.startswith('*Nset'):
                element = False
                bc = 'nset=Dirichlet' in lines
            if bc:
                bc_enc.append(lines)
            if 'elset=Dirichlet' in lines:
                bc = False
            if lines.startswith('*Nset'):
                ld = 'nset=Set-4' in lines
            if ld:
                bc_load.append(lines)
            if 'elset=Set-4' in lines:
                ld = False

    node_list0 = list_to_array(node_list[1:-1], None)[:, 1:-1]
    element_list0 = list_to_array(element_list[1:-1], None)[:, 1:]
    bc_enc0 = list_to_array(bc_enc[1:-1], 'bc')
    bc_load0 = list_to_array(bc_load[1:-1], 'bc')
    
    return node_list0, element_list0, bc_enc0, bc_load0, f_attr

In [6]:
def cell_connectivity(n_pts, element_list0):
    
    conn0 = np.zeros((n_pts, 4))

    for cell in range(element_list0.shape[0]):
        for point_id in range(element_list0.shape[1]):
            pt = int(element_list0[cell][point_id])
            
            for idx in range(4):
                if conn0[pt-1, idx] == 0:
                    conn0[pt-1, idx] = cell + 1
                    break

    return conn0

In [7]:
def bound_cond(bc, n_pts, l_type, *load):
    bc0 = np.zeros((n_pts, 1))

    if l_type == 'bc':
        for idx in range(bc.shape[0]):
            pt = int(bc[idx])
            bc0[pt - 1] = 999999

    elif l_type == 'load':
        for idx in range(bc.shape[0]):
            pt = int(bc[idx])
            bc0[pt - 1] = load

    return bc0

In [8]:
def get_load(l_type, load_idx):
    loads = {'Tensile': np.arange(0.02, 0.17, 0.02),
          'Compression': -np.arange(0.02, 0.17, 0.02),
          'Cyclic': np.arange(0.02, 0.17, 0.02)}
    
    return loads[l_type][load_idx]

In [9]:
def file_attr(filepath):

    f_attr = {}
    f1 = filepath.split('\\')
    f_idx = f1.index('VTK_Data')
    f2 = f1[f_idx+1:]

    load_idx0 = re.findall('J\d+', f2[5])
    load_idx = int(str(load_idx0[0]).split('J')[-1])

    f_attr.update(plate_type = f2[0],
                  material = f2[1],
                  nlgeom = f2[2],
                  mat_b = f2[3],
                  l_type = f2[4],
                  load_idx = load_idx)
    
    return f_attr

In [10]:
def get_max_n_pts(fp_dir):
    
    global max_n_pts
    # max_n_pts = 0
    # fields = {}

    # for f in glob.glob(fp_dir+'/**/*1.vtu', recursive = True):
        
    #     file = pv.read(f)
    #     if max_n_pts == 0:
    #         for i in range(len(file.point_data.keys())):
    #             fields[i] = file.point_data.keys()[i]
    #         print('Available data:')
    #         for key, value in fields.items():
    #             print('{:<3d} : {:<35s} Shape : {:<8s}'.format(key, value, str(file.point_data.get_array(key).shape)))

    #     if file.n_points > max_n_pts:
    #         max_n_pts = file.n_points

    max_n_pts = 625
    
    return max_n_pts

In [11]:
def ragged(f_tensor):

    f_tensor_ragged = tf.zeros((1, 25, 25, f_tensor.shape[-1]), dtype = f_tensor.dtype)

    for i in range (f_tensor.shape[0]):
        f_dummy = tf.RaggedTensor.from_tensor(f_tensor[i, :, :, :], padding = tf.constant(value = 0, shape = (f_tensor.shape[-1]), dtype = f_tensor.dtype))
        f_dummy = tf.expand_dims(f_dummy, axis = 0)
        f_tensor_ragged = tf.concat([f_tensor_ragged, f_dummy], axis = 0)

    f_tensor_ragged = f_tensor_ragged[1:, :, :, :]

    return f_tensor_ragged

In [12]:
def generate_samples(generator, fip, fop, f_attr_dict, n_samples):
    
    idx = np.random.randint(0, fip_tensor.shape[0], n_samples)
    X_cond = tf.zeros((1, 6), dtype = tf.float64)
    loads = {'Tensile': np.arange(0.02, 0.17, 0.02),
          'Compression': -np.arange(0.02, 0.17, 0.02),
          'Cyclic': np.arange(0.02, 0.17, 0.02)}

    #square = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]

    for i in range(len(idx)):
        
        f_att = f_attr_dict[idx[i]]
        
        if f_att['plate_type'] == 'With_Hole':
            ptype = 0

        if f_att['material'] == 'Aluminium':
            mat = 70
        
        if f_att['nlgeom'] == 'NLGEOM_ON':
            nl = 1
        elif f_att['nlgeom'] == 'NLGEOM_OFF':
            nl = 0
        
        if f_att['mat_b'] == 'Elastic':
            mbeh = 0
        elif f_att['mat_b'] == 'Visco-Plastic':
            mbeh = 1

        if f_att['l_type'] == 'Tensile':
            ltype = 1
        elif f_att['l_type'] == 'Compression':
            ltype = -1

        ld = loads[f_att['l_type']][f_att['load_idx']]
        #slen = 2 * ld
        cond = [ptype, mat, nl, mbeh, ltype, ld]
        cond = tf.expand_dims(tf.convert_to_tensor(cond, dtype = tf.float64), axis = 0)
        X_cond = tf.concat([X_cond, cond], axis = 0)
        
    X_cond = X_cond[1:, :]
    idx = [[i] for i in idx]

    X_undef = tf.gather_nd(fip, idx)
    Y_def = tf.gather_nd(fop, idx)
    gen_def = generator([X_undef, X_cond])
   
    return [X_undef, X_cond, gen_def, Y_def]

In [13]:
def read_file(filepath):
    
    if filepath.endswith('0.vtu'):
        node_list0, element_list0, bc_enc0, bc_load0, f_attr = get_inp(filepath)
        n_pts = node_list0.shape[0]
        conn0 = cell_connectivity(n_pts, element_list0)
        
        load = get_load(f_attr['l_type'], f_attr['load_idx'])
        bc_enc1 = bound_cond(bc_enc0, n_pts, 'bc')
        bc_load1 = bound_cond(bc_load0, n_pts, 'load', load)
        
        f_tensor = tf.concat((node_list0, conn0, bc_enc1, bc_load1), axis = 1)
        paddings = tf.constant([[0, max_n_pts-f_tensor.shape[0],], [0, 0]])
        f_tensor = tf.pad(f_tensor, paddings, "CONSTANT", constant_values = 0)
        f_tensor = tf.expand_dims(f_tensor, axis = 0)

        return f_tensor, f_attr

    else:
        vtu_file0 = pv.read(filepath)
        crd = vtu_file0.points[:, :-1]
        # stress_cmp = vtu_file0.point_data.get_array('Stress_Components')
        # strain_cmp = vtu_file0.point_data.get_array('Logarithmic_strain_components')
        stress_mises = tf.expand_dims(vtu_file0.point_data.get_array('Stress_Mises'), axis = -1)

        f_tensor = tf.concat((crd, stress_mises), axis = 1)
        paddings = tf.constant([[0, max_n_pts-f_tensor.shape[0],], [0, 0]])
        f_tensor = tf.pad(f_tensor, paddings, "CONSTANT", constant_values = 0)
        f_tensor = tf.expand_dims(f_tensor, axis =0)
        
        return f_tensor, _

In [14]:
def load_real_data():

    get_max_n_pts(fp_vtk)
    fip_tensor = tf.zeros((1, max_n_pts, 8), dtype = tf.float64)
    f_attr_dict = {}
    file_count = 0
    fop_tensor = tf.zeros((1, max_n_pts, 3), dtype = tf.float32)

    for filepath in glob.glob(fp_vtk+'/**/*.vtu', recursive = True):
        
        if filepath.endswith('0.vtu'):
            f_tensor, f_attr = read_file(filepath)
        else:
            f_tensor, _ = read_file(filepath)

        if filepath.endswith('0.vtu'):
            fip_tensor = tf.concat([fip_tensor, f_tensor], axis = 0)
            f_attr_dict[file_count] = f_attr
            print('fip:', fip_tensor.shape)
            file_count += 1
            
        else:
            fop_tensor = tf.concat([fop_tensor, f_tensor], axis = 0)
            print('fop:', fop_tensor.shape)

    fip_tensor = fip_tensor[1:, :, :]
    fop_tensor = fop_tensor[1:, :, :]

    # a, _ = tf.linalg.normalize(fop_tensor[:, :, -1])
    a_max = tf.reduce_max(tf.reshape(fop_tensor[:, :, -1], -1)) # 2455897900.0
    a_norm = fop_tensor[:, :, -1] / a_max
    a_norm = tf.expand_dims(a_norm, axis = -1)
    fop_tensor = tf.concat([fop_tensor[:, :, :-1], a_norm], axis = -1)

    fip_tensor = tf.reshape(fip_tensor, shape = (-1, 25, 25, 8))
    fop_tensor = tf.reshape(fop_tensor, shape = (-1, 25, 25, 3))
    
    # fip_tensor = ragged(fip_tensor)
    # fop_tensor = ragged(fop_tensor)

    return fip_tensor, fop_tensor, f_attr_dict

# Generator:

In [15]:
def BatchNorm_Act(x, act=True):
    
    x = tf.keras.layers.BatchNormalization()(x)
    
    if act == True:
        x = tf.keras.layers.Activation('elu')(x)
        
    return x

In [16]:
def conv_block(x, filters, act_fn, kernel_size, strides):
    
    conv = Conv2D(filters, kernel_size=kernel_size, padding='same', strides=strides, kernel_initializer = ker_init)(x)
    conv = BatchNorm_Act(conv)
    
    return conv

In [17]:
def initial_block(x, filters, act_fn):
    
    conv = Conv2D(filters, kernel_size=3, padding='same', strides=1, kernel_initializer = ker_init)(x)
    conv = conv_block(conv, filters, act_fn, kernel_size=3, strides=1)
    
    sc = Conv2D(filters, kernel_size=1, padding='same', strides=1, kernel_initializer = ker_init)(x)
    sc = BatchNorm_Act(sc, act=False)
    
    output = Add()([conv, sc])
    
    return output

In [18]:
def res_block_2(x, filters, act_fn, s_val):
    
    res = conv_block(x, filters/4, act_fn, kernel_size=1, strides=1)
    res = conv_block(res, filters/4, act_fn, kernel_size=3, strides=s_val)
    res = conv_block(res, filters, act_fn, kernel_size=1, strides=1)
    
    sc = Conv2D(filters, kernel_size=1, padding='same', strides=s_val, kernel_initializer = ker_init)(x)
    sc = BatchNorm_Act(sc, act=False)
    
    output = Add()([sc, res])
    
    return output

In [19]:
def upsample_block(x, skip_feature, filters, act_fn):
    
    upsample = Conv2DTranspose(filters, kernel_size=3, padding='same', strides=2, kernel_initializer = ker_init) (x)
    concat = concatenate([upsample, skip_feature])
    
    return concat

In [20]:
def generator():
    
    seed = 1
    activ = "elu"
    global ker_init
    ker_init = keras.initializers.he_normal(seed=seed)
    in_shape = (25, 25, 8)
    cond_shape = (6,)
    n_fact = 16
    f = [64, 128, 256, 512]
    
    gen_inp0 = Input(in_shape, name = 'generator_input')
    print('gen_inp shape:', gen_inp0.shape)
    gen_inp1 = Dense(n_fact*n_fact*in_shape[2], kernel_initializer = ker_init) (gen_inp0)
    gen_inp = Reshape((n_fact*in_shape[0], n_fact*in_shape[1], in_shape[2])) (gen_inp1)
    print('reshaped input:', gen_inp.shape)
    
    enc_1 = initial_block(gen_inp, f[0], activ)
    print('enc_1 shape:', enc_1.shape)
    
    enc_2 = res_block_2(enc_1, f[1], activ, s_val = 2)
    print('enc_2 shape:', enc_2.shape)
    
    enc_3 = res_block_2(enc_2, f[2], activ, s_val = 2)
    print('enc_3 shape:', enc_3.shape)
    
    br_1 = conv_block(enc_3, f[3], activ, kernel_size = 3, strides = 2)
    print('br_1 shape:', br_1.shape)
    
    cond0 = Input(cond_shape, name = 'condition_input')
    cond1 = Dense(br_1.shape[1]*br_1.shape[2]*br_1.shape[3], kernel_initializer = ker_init) (cond0)
    cond = Reshape(br_1.shape[1:]) (cond1)

    br_1 = concatenate([br_1, cond])
    print('concat shape:', br_1.shape)

    br_2 = conv_block(br_1, f[3], activ, kernel_size = 3, strides = 1)
    print('br_2 shape:', br_2.shape)
    
    up_1 = upsample_block(br_2, enc_3, f[2], activ)
    dec_1 = res_block_2(up_1, f[2], activ, s_val = 1)
    print('dec_1 shape:', dec_1.shape)
    
    up_2 = upsample_block(dec_1, enc_2, f[1], activ)
    dec_2 = res_block_2(up_2, f[1], activ, s_val = 1)
    print('dec_2 shape:', dec_2.shape)
    
    up_3 = upsample_block(dec_2, enc_1, f[0], activ)
    dec_3 = res_block_2(up_3, f[0], activ, s_val=n_fact)
    print('dec_3 shape:', dec_3.shape)
    
    outputs = Conv2D(3, 1, padding='same', strides = 1, activation='tanh')(dec_3)
    print('output shape:', outputs.shape)
    
    model = Model(inputs=[gen_inp0, cond0], outputs=outputs)
    print('Generator')
    
    return model

# Discriminator:

In [21]:
def discriminator():
    
    activ = "elu"
    f = [64, 128, 256, 512]
    n_fact = 16
    in_shape = (25, 25, 8)
    tar_shape = (25, 25, 3)

    disc_inp = Input(in_shape, name = 'def_tensor from gen')
    print('disc_inp:', disc_inp.shape)
    # disc_inp = Dense(n_fact*n_fact*in_shape[2], activation = 'relu', kernel_initializer = ker_init) (disc_inp)
    # disc_inp = Reshape((in_shape[0]*n_fact, in_shape[1]*n_fact, in_shape[2])) (disc_inp)
    # print('reshaped input:', disc_inp.shape)
    
    disc_tar = Input(tar_shape, ragged = True, name = 'actual def_tensor')
    print('disc_tar:', disc_tar.shape)
    # disc_tar = Dense(n_fact*n_fact*tar_shape[2], activation = activ, kernel_initializer = ker_init) (disc_tar)
    # disc_tar = Reshape((tar_shape[0]*n_fact, tar_shape[1]*n_fact, tar_shape[2])) (disc_tar)
    
    disc_concat = concatenate([disc_inp, disc_tar])
    print('disc_concat:', disc_concat.shape)
    
    disc_concat1 = Dense(n_fact*n_fact*disc_concat.shape[3], activation = 'relu', kernel_initializer = ker_init) (disc_concat)
    disc_concat = Reshape((disc_concat.shape[1]*n_fact, disc_concat.shape[2]*n_fact, disc_concat.shape[3])) (disc_concat1)
    print('reshaped input:', disc_concat.shape)
    
    conv = conv_block(disc_concat, f[0], activ, kernel_size=3, strides=2)
    print(conv.shape)
    conv = conv_block(conv, f[1], activ, kernel_size=3, strides=2)
    print(conv.shape)
    
    conv = conv_block(conv, f[2], activ, kernel_size=3, strides=2)
    print(conv.shape)
    conv = conv_block(conv, f[2], activ, kernel_size=3, strides=2)
    print(conv.shape)

    zero_pad1 = tf.keras.layers.ZeroPadding2D()(conv)
    conv = conv_block(zero_pad1, f[3], activ, kernel_size=3, strides=2)
    print(conv.shape)

    conv = Conv2D(f[3], 3, strides = 1, kernel_initializer = ker_init)(conv)
    print(conv.shape)
    conv = BatchNorm_Act(conv)

    outputs = Conv2D(1, 3, strides = 1, kernel_initializer = ker_init)(conv)
    print('output shape:', outputs.shape)

    model = Model(inputs = [disc_inp, disc_tar], outputs = outputs)
    print('Discriminator')
    
    return model

# GAN model:

In [22]:
optimizer_gen = Adam(learning_rate = 0.0001)
optimizer_disc = Adam(learning_rate = 0.001)

In [39]:
@tf.function
def train(generator, discriminator, fip, fop, f_attr_dict, n_epochs, n_batch):

    bat_per_epo = int(fip.shape[0] / n_batch)
    LAMBDA = 100
    
    for i in range(n_epochs):

        for j in range(bat_per_epo):

            [X_undef, X_cond, gen_def, Y_def] = generate_samples(generator, fip, fop, f_attr_dict, n_batch)
            print('X_undef:', X_undef.shape, 'X_cond:', X_cond.shape, 'gen_def:', gen_def.shape, 'Y_def:', Y_def.shape)
            
            # d_real_loss = discriminator.train_on_batch([X_undef, Y_def], tf.ones_like((10, 1), dtype = tf.float32))
            # d_fake_loss = discriminator.train_on_batch([X_undef, gen_def], tf.zeros_like((10, 1), dtype = tf.float32))
            # d_loss = (d_real_loss + d_fake_loss) / 2.0

            # g_loss = generator.train_on_batch([X_undef, X_cond], tf.ones_like(Y_def, dtype = tf.float32))
            # print(d_real_loss, d_fake_loss, g_loss)
            with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
                
                d_real = discriminator([X_undef, Y_def], training = True)
                d_fake = discriminator([X_undef, gen_def], training = True)
                
                d_real_loss = tf.keras.losses.MeanSquaredError()(tf.ones_like(d_real), d_real) # check for ragged error
                d_fake_loss = tf.keras.losses.MeanSquaredError()(tf.zeros_like(d_fake), d_fake)
                d_loss = (d_real_loss + d_fake_loss) / 2.0
                print(d_loss)
                g_loss = tf.keras.losses.MeanSquaredError()(tf.ones_like(gen_def), gen_def)
                g_l1_loss = tf.reduce_mean(tf.abs(Y_def - gen_def))
                total_g_loss = g_loss #+ (LAMBDA * g_l1_loss)
                print(total_g_loss)
            print(generator.trainable_variables)
            print(discriminator.trainable_variables)
            generator_gradients = gen_tape.gradient(total_g_loss, generator.trainable_variables)
            discriminator_gradients = disc_tape.gradient(d_loss, discriminator.trainable_variables)
            
            optimizer_gen.apply_gradients(zip(generator_gradients, generator.trainable_variables))
            optimizer_disc.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))
            
            print(i, j)
            # with summary_writer.as_default():
            #     tf.summary.scalar('total_g_loss', total_g_loss, step = i//100)
            #     tf.summary.scalar('g_loss', g_loss, step = i//100)
            #     tf.summary.scalar('g_l1_loss', g_l1_loss, step = i//100)
            #     tf.summary.scalar('d_loss', d_loss, step = i//100)
        
        if (n_epochs + 1) % 10 == 0:
            print('.', end='', flush=True)

        if (n_epochs + 1) % 500 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

# Main Program:

In [24]:
fip_tensor, fop_tensor, f_attr_dict = load_real_data()

fip: (2, 625, 8)
fop: (2, 625, 3)
fip: (3, 625, 8)
fop: (3, 625, 3)
fip: (4, 625, 8)
fop: (4, 625, 3)
fip: (5, 625, 8)
fop: (5, 625, 3)
fip: (6, 625, 8)
fop: (6, 625, 3)
fip: (7, 625, 8)
fop: (7, 625, 3)
fip: (8, 625, 8)
fop: (8, 625, 3)
fip: (9, 625, 8)
fop: (9, 625, 3)
fip: (10, 625, 8)
fop: (10, 625, 3)
fip: (11, 625, 8)
fop: (11, 625, 3)
fip: (12, 625, 8)
fop: (12, 625, 3)
fip: (13, 625, 8)
fop: (13, 625, 3)
fip: (14, 625, 8)
fop: (14, 625, 3)
fip: (15, 625, 8)
fop: (15, 625, 3)
fip: (16, 625, 8)
fop: (16, 625, 3)
fip: (17, 625, 8)
fop: (17, 625, 3)
fip: (18, 625, 8)
fop: (18, 625, 3)
fip: (19, 625, 8)
fop: (19, 625, 3)
fip: (20, 625, 8)
fop: (20, 625, 3)
fip: (21, 625, 8)
fop: (21, 625, 3)
fip: (22, 625, 8)
fop: (22, 625, 3)
fip: (23, 625, 8)
fop: (23, 625, 3)
fip: (24, 625, 8)
fop: (24, 625, 3)
fip: (25, 625, 8)
fop: (25, 625, 3)
fip: (26, 625, 8)
fop: (26, 625, 3)
fip: (27, 625, 8)
fop: (27, 625, 3)
fip: (28, 625, 8)
fop: (28, 625, 3)
fip: (29, 625, 8)
fop: (29, 625, 3)
fip: (30

In [25]:
gen = generator()
gen.summary()

disc = discriminator()
disc.summary()

gen_inp shape: (None, 25, 25, 8)
reshaped input: (None, 400, 400, 8)
enc_1 shape: (None, 400, 400, 64)
enc_2 shape: (None, 200, 200, 128)
enc_3 shape: (None, 100, 100, 256)
br_1 shape: (None, 50, 50, 512)
concat shape: (None, 50, 50, 1024)
br_2 shape: (None, 50, 50, 512)
dec_1 shape: (None, 100, 100, 256)
dec_2 shape: (None, 200, 200, 128)
dec_3 shape: (None, 25, 25, 64)
output shape: (None, 25, 25, 3)
Generator
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 generator_input (InputLayer)   [(None, 25, 25, 8)]  0           []                               
                                                                                                  
 dense (Dense)                  (None, 25, 25, 2048  18432       ['generator_input[0][0]']        
                                )                                         

In [26]:
# disc.compile(loss = tf.keras.losses.MeanSquaredError(), optimizer = optimizer_disc)
# gen.compile(loss = tf.keras.losses.MeanSquaredError(), optimizer = optimizer_gen)

In [27]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer = optimizer_gen,
                                 discriminator_optimizer = optimizer_disc,
                                 generator=gen,
                                 discriminator=disc)

In [28]:
log_dir="logs/"

summary_writer = tf.summary.create_file_writer(
  log_dir + "FEGAN/" + datetime.datetime.now().strftime("%d-%m-%Y_%H-%M-%S"))

In [29]:
# %load_ext tensorboard
# %tensorboard --logdir {log_dir}

In [38]:
train(gen, disc, fip_tensor, fop_tensor, f_attr_dict, 1000, 2)

X_undef: (2, 25, 25, 8) X_cond: (2, 6) gen_def: (2, 25, 25, 3) Y_def: (2, 25, 25, 3)
Tensor("truediv:0", shape=(), dtype=float32)


TypeError: in user code:

    File "C:\Users\VAISHN~1\AppData\Local\Temp/ipykernel_4120/2509801169.py", line 29, in train  *
        g_loss = tf.keras.losses.mean_squared_error()(tf.ones_like(gen_def), gen_def)

    TypeError: Missing required positional argument


In [ ]:
gen.predict(fip_tensor[0:5, :, :, :])

ValueError: Failed to find data adapter that can handle input: <class 'method'>, <class 'NoneType'>

In [ ]:
i = Input(shape=(5,5,3), ragged = True)
c = Conv2D(64, (3,3), padding="same")(i)
m = Model(inputs=i,outputs=c)

In [ ]:
m(fop_tensor[0:2,0:5,0:5,:])

ValueError: Exception encountered when calling layer "conv2d_285" "                 f"(type Conv2D).

TypeError: object of type 'RaggedTensor' has no len()


Call arguments received by layer "conv2d_285" "                 f"(type Conv2D):
  • inputs=<tf.RaggedTensor [[[[-0.33118883, 0.30870724, 0.00015755338],
   [-0.30863988, 0.3314005, 0.0003424207],
   [-0.32586306, 0.3536344, 0.0003726331],
   [-0.35792002, 0.32751453, 0.00020738765],
   [-0.27821872, 0.34635574, 0.0005788191]],

  [[-0.44693792, 0.4523728, 0.0004224972],
   [-0.5, 0.5, 0.0004916506],
   [-0.5, 0.45125842, 0.0004139827],
   [-0.44667, 0.4034717, 0.00032260435],
   [-0.5, 0.40185198, 0.0002893679]],

  [[0.498, 0.30664742, 0.0002680732],
   [0.45100728, 0.30858254, 0.00026780064],
   [0.498, 0.35507628, 0.0002655448],
   [0.45029017, 0.35628536, 0.00026542213],
   [0.40821108, 0.16826282, 0.00027676753]],

  [[0.26685828, 0.27796406, 0.00026096572],
   [0.26323825, 0.32304, 0.00026119844],
   [0.25982168, 0.367753, 

In [ ]:
#os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'
tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [ ]:
# change values for cond
# change func for generalized filepath